<a href="https://colab.research.google.com/github/aitch-cmd/FootfallTracker/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ultralytics

In [ ]:
# Importing the YOLO package from the ultralytics library to use YOLOv8
from ultralytics import YOLO

In [ ]:
# Initializing the YOLOv8 model using a pre-trained weight file (YOLOv8n - nano version)
model = YOLO('yolov8n.pt')

In [ ]:
!pip install -q --upgrade supervision

### Making annotations

In [ ]:
import supervision as sv

# Specify the input video path
video_path = "/content/drive/MyDrive/testing-1.mp4"

# Create the video frames generator
generator = sv.get_video_frames_generator(video_path)

# Create an iterator and get the first frame
iterator = iter(generator)
frame = next(iterator)

# Perform inference on the first frame
results = model(frame, imgsz=1280)[0]

# Convert inference results to Detections
detections = sv.Detections.from_ultralytics(results)

# Create the correct BoxAnnotator object
box_annotator = sv.BoxAnnotator(thickness=4)

# Annotate the frame with bounding boxes
frame = box_annotator.annotate(scene=frame, detections=detections)

# Display the annotated frame
%matplotlib inline
sv.plot_image(frame, (16, 16))

### Adding grids

In [ ]:
import supervision as sv
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Specify the video path directly
video_path = "/content/drive/MyDrive/testing-1.mp4"  # Replace with the actual path to your video

# Create the video frames generator
generator = sv.get_video_frames_generator(video_path)

# Create an iterator and get the first frame
iterator = iter(generator)
frame = next(iterator)

# Define a single zone as a rectangle (x1, y1, x2, y2)
zone = {"x1": 100, "y1": 100, "x2": 500, "y2": 300}
#x1, y1 represent the top-left corner of the rectangle.
#x2, y2 represent the bottom-right corner of the rectangle.

# Draw the zone on the frame
x1, y1, x2, y2 = zone["x1"], zone["y1"], zone["x2"], zone["y2"]
cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw a rectangle for the zone

# Convert frame to RGB for Matplotlib
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Display the frame with the zone
plt.figure(figsize=(10, 10))
plt.imshow(frame_rgb)
plt.title("Frame with Zone")
plt.grid(color='white', linestyle='--', linewidth=0.5)

# Add grid lines for measurement
plt.xticks(np.arange(0, frame_rgb.shape[1], 100))
plt.yticks(np.arange(0, frame_rgb.shape[0], 100))

plt.show()

In [ ]:
import supervision as sv
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO

# Specify the video path directly
video_path = "/content/drive/MyDrive/testing-1.mp4"

model = YOLO('yolov8n.pt')  # or yolov8s.pt, yolov8m.pt etc.

# Create the video frames generator
generator = sv.get_video_frames_generator(video_path)

# Create an iterator and get the first frame
iterator = iter(generator)
frame = next(iterator)

# Perform inference on the first frame
results = model(frame)[0]
detections = sv.Detections.from_ultralytics(results)

# Annotate only bounding boxes (skip label annotation)
box_annotator = sv.BoxAnnotator(thickness=4)
annotated_frame = box_annotator.annotate(scene=frame, detections=detections)

# Define a single zone as a rectangle (x1, y1, x2, y2)
zone = {"x1": 100, "y1": 100, "x2": 500, "y2": 300}

# Draw the zone on the frame
x1, y1, x2, y2 = zone["x1"], zone["y1"], zone["x2"], zone["y2"]
cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw a rectangle for the zone

# Convert the frame to RGB format for Matplotlib (if needed)
annotated_frame_rgb = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)

# Plot the annotated frame with grid lines
plt.figure(figsize=(10, 10))
plt.imshow(annotated_frame_rgb)
plt.title("Annotated Frame with Zone and Detections")

# Add grid lines
plt.grid(color='white', linestyle='--', linewidth=0.5)

# Set grid properties (optional, e.g., based on pixels)
plt.xticks(np.arange(0, annotated_frame_rgb.shape[1], 100))  # Set x-ticks (every 100 pixels)
plt.yticks(np.arange(0, annotated_frame_rgb.shape[0], 100))  # Set y-ticks (every 100 pixels)

plt.show()

Testing

Dwell Time Tracking

In [ ]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
import supervision as sv

def process_video_return_last_frame(video_path, output_path):
    model = YOLO('yolov8n.pt')  # Load YOLOv8 model

    # Define polygon zone
    polygon = np.array([[100, 100], [500, 100], [500, 400], [100, 400]])
    zone = sv.PolygonZone(polygon=polygon)
    zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color(255, 0, 0))
    box_annotator = sv.BoxAnnotator(color=sv.Color(138, 43, 226), thickness=2)

    # Video reading
    cap = cv2.VideoCapture(video_path)
    width, height = int(cap.get(3)), int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Heatmap init
    heatmap = np.zeros((height, width), dtype=np.float32)
    track_memory = {}  # track_id: (cx, cy, time_entered)

    # Writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    last_frame = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        now = time.time()

        results = model.track(frame, persist=True)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[detections.class_id == 0]  # Only persons

        for i, xyxy in enumerate(detections.xyxy):
            if i >= len(detections.tracker_id):
                continue

            track_id = detections.tracker_id[i]
            x1, y1, x2, y2 = map(int, xyxy)
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            if cv2.pointPolygonTest(polygon, (cx, cy), False) >= 0:
                if track_id in track_memory:
                    old_cx, old_cy, time_entered = track_memory[track_id]
                    dist = np.hypot(cx - old_cx, cy - old_cy)

                    if dist < 10:
                        dwell_time = now - time_entered
                        if dwell_time > 5:
                            cv2.circle(heatmap, (cx, cy), radius=15, color=5, thickness=-1)
                        else:
                            cv2.circle(heatmap, (cx, cy), radius=15, color=1, thickness=-1)
                    else:
                        track_memory[track_id] = (cx, cy, now)
                        cv2.circle(heatmap, (cx, cy), radius=15, color=1, thickness=-1)
                else:
                    track_memory[track_id] = (cx, cy, now)
                    cv2.circle(heatmap, (cx, cy), radius=15, color=1, thickness=-1)

        # Annotate
        annotated = frame.copy()
        annotated = zone_annotator.annotate(scene=annotated)
        annotated = box_annotator.annotate(scene=annotated, detections=detections)

        # Heatmap overlay
        blurred = cv2.GaussianBlur(heatmap, (31, 31), 0)
        normalized = cv2.normalize(blurred, None, 0, 255, cv2.NORM_MINMAX)
        heatmap_colored = cv2.applyColorMap(normalized.astype(np.uint8), cv2.COLORMAP_JET)
        overlay = cv2.addWeighted(annotated, 0.7, heatmap_colored, 0.3, 0)


        # Count how many people are in the zone
        in_zone_mask = zone.trigger(detections=detections)
        people_in_zone = np.sum(in_zone_mask)

        # Draw live count
        # cv2.putText(
        #     overlay,
        #     f'People in Zone: {int(people_in_zone)}',
        #     (50, 50),
        #     cv2.FONT_HERSHEY_SIMPLEX,
        #     1,
        #     (0, 255, 0),
        #     2
        # )

        out.write(overlay)
        last_frame = overlay.copy()

    cap.release()
    out.release()

    return output_path, last_frame


In [ ]:
#Ask user for input and output file paths
video_path = input("Enter path to input video: ").strip()
output_path = input("Enter path to save output video: ").strip()

#Process video
final_video_path, last_frame = process_video_return_last_frame(video_path, output_path)

#Display the last frame using matplotlib
plt.imshow(cv2.cvtColor(last_frame, cv2.COLOR_BGR2RGB))
plt.title("Last Frame of Output Video")
plt.axis("off")
plt.show()

In [ ]:
video_path = "/content/drive/MyDrive/testing-1.mp4"
output_path = "/content/drive/MyDrive/testdemo01.mp4"

# Process video and get last frame
final_video_path, last_frame = process_video_return_last_frame(video_path, output_path)

# Display the last frame
import matplotlib.pyplot as plt
plt.imshow(cv2.cvtColor(last_frame, cv2.COLOR_BGR2RGB))
plt.title("Last Frame of Output Video")
plt.axis("off")
plt.show()
